In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as func
import torch.nn.init as torch_init
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from torch.utils.data.sampler import SubsetRandomSampler
import numpy as np
import TIMIT_dataloader
from tqdm import tqdm_notebook as tqdm
from Model import autoencoder

num_epochs = 10           # Number of full passes through the dataset
batch_size = 50          # Number of samples in each minibatch
num_frame = 11
use_cuda = torch.cuda.is_available()

# Setup GPU optimization if CUDA is supported
if use_cuda:
    computing_device = torch.device("cuda")
    extras = {"num_workers": 1, "pin_memory": True}
    print("CUDA is supported")
else: # Otherwise, train on the CPU
    computing_device = torch.device("cpu")
    extras = False
    print("CUDA NOT supported")

train_loader = TIMIT_dataloader.prepareTIMIT_train(batch_size=batch_size,
                                           num_frame=num_frame,
                                           extras=False)

print('train_loader complete')
model = autoencoder().cuda()
model = model.to(computing_device)
print("Model on CUDA?", next(model.parameters()).is_cuda)

criterion = nn.MSELoss()

learning_rate = 0.1
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

loss_list = []
"""
for epoch in range(num_epochs):

    N = 50
    # Get the next minibatch of images, labels for training
    #for minibatch_count, (inputs,target) in enumerate(tqdm(train_loader), 0):
    for minibatch_count, (inputs, target) in enumerate(tqdm(train_loader), 0):


        # Put the minibatch data in CUDA Tensors and run on the GPU if supported
        inputs, target = inputs.to(computing_device), target.to(computing_device)

        # Zero out the stored gradient (buffer) from the previous iteration
        optimizer.zero_grad()

        # Perform the forward pass through the network and compute the loss
        outputs = model(inputs)
        loss = criterion(outputs, target)

        # Automagically compute the gradients and backpropagate the loss through the network
        loss.backward()

        # Update the weights
        optimizer.step()

        # Add this iteration's loss to the total_loss
        

        if minibatch_count % N == 0:
            loss_list.append(loss)
            print('Epoch {}, Minibatch {}, Training Loss {}'.format(epoch, minibatch_count, loss))
            print('target',target.shape)
            print('output',outputs)
"""



CUDA is supported
train_loader complete
Model on CUDA? True


"\nfor epoch in range(num_epochs):\n\n    N = 50\n    # Get the next minibatch of images, labels for training\n    #for minibatch_count, (inputs,target) in enumerate(tqdm(train_loader), 0):\n    for minibatch_count, (inputs, target) in enumerate(tqdm(train_loader), 0):\n\n\n        # Put the minibatch data in CUDA Tensors and run on the GPU if supported\n        inputs, target = inputs.to(computing_device), target.to(computing_device)\n\n        # Zero out the stored gradient (buffer) from the previous iteration\n        optimizer.zero_grad()\n\n        # Perform the forward pass through the network and compute the loss\n        outputs = model(inputs)\n        loss = criterion(outputs, target)\n\n        # Automagically compute the gradients and backpropagate the loss through the network\n        loss.backward()\n\n        # Update the weights\n        optimizer.step()\n\n        # Add this iteration's loss to the total_loss\n        \n\n        if minibatch_count % N == 0:\n         

In [1]:
import numpy as np
import torch
import torch.utils.data
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from torchvision import datasets, transforms
from torchvision.utils import make_grid , save_image

In [55]:
class RBM(nn.Module):
    def __init__(self,
                 n_vis=1419,
                 n_hin=500,
                 k=5):
        super(RBM, self).__init__()
        self.W = nn.Parameter(torch.randn(n_hin,n_vis)*1e-2)
        self.v_bias = nn.Parameter(torch.zeros(n_vis))
        self.h_bias = nn.Parameter(torch.zeros(n_hin))
        self.k = k
    
    def sample_from_p(self,p):
        return F.relu(torch.sign(p - Variable(torch.rand(p.size())).cuda()))
    
    def v_to_h(self,v):
        p_h = F.sigmoid(F.linear(v,self.W,self.h_bias))
        sample_h = self.sample_from_p(p_h)
        return p_h,sample_h
    
    def h_to_v(self,h):
        p_v = F.sigmoid(F.linear(h,self.W.t(),self.v_bias))
        sample_v = self.sample_from_p(p_v)
        return p_v,sample_v
        
    def forward(self,v):
        pre_h1,h1 = self.v_to_h(v)
        
        h_ = h1
        for _ in range(self.k):
            pre_v_,v_ = self.h_to_v(h_)
            pre_h_,h_ = self.v_to_h(v_)
        
        return v,v_
    
    def free_energy(self,v):
        vbias_term = v.mv(self.v_bias)
        wx_b = F.linear(v,self.W,self.h_bias)
        hidden_term = wx_b.exp().add(1).log().sum(1)
        return (-hidden_term - vbias_term).mean()

In [56]:
batch_size = 64
train_loader = TIMIT_dataloader.prepareTIMIT_train(batch_size=batch_size,
                                           num_frame=num_frame,
                                           extras=False)

In [57]:
rbm = RBM(k=1)

In [58]:
rbm=rbm.to(computing_device)

In [59]:
computing_device = torch.device("cuda")

In [60]:
train_op = optim.SGD(rbm.parameters(),0.1)

In [52]:
type(sample_data)

torch.Tensor

In [66]:
for epoch in range(10):
    loss_ = []
    for _, (data,target) in enumerate(train_loader):
        data = Variable(data.view(-1,1419))
        print(data.shape)
        #sample_data = data.bernoulli(0.9)
        sample_data=sample_data.to(computing_device)
        #print(type(sample_data))
        #sample_data=sample_data.cuda()
        v,v1 = rbm(sample_data)
        loss = rbm.free_energy(v) - rbm.free_energy(v1)
        print(loss.data.item())
        loss_.append(loss.data.item())
        train_op.zero_grad()
        loss.backward()
        train_op.step()
    
    print(np.mean(loss_))

torch.Size([64, 1419])


D:\Users\ericl\Anaconda3\lib\site-packages\torch\nn\functional.py:1332: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


1.676025390625
torch.Size([64, 1419])
-0.001953125
torch.Size([64, 1419])
2.095703125
torch.Size([64, 1419])
2.839111328125
torch.Size([64, 1419])
3.305419921875
torch.Size([64, 1419])
-2.43115234375
torch.Size([64, 1419])
6.34228515625
torch.Size([64, 1419])
-2.745849609375
torch.Size([64, 1419])
-5.117919921875
torch.Size([64, 1419])
-3.60986328125
torch.Size([64, 1419])
-7.02392578125
torch.Size([64, 1419])
-0.548828125
torch.Size([64, 1419])
-9.552490234375
torch.Size([64, 1419])
-2.3935546875
torch.Size([64, 1419])
1.180419921875
torch.Size([64, 1419])
-4.507080078125
torch.Size([64, 1419])
-1.486083984375
torch.Size([64, 1419])
-4.05517578125
torch.Size([64, 1419])
-6.41943359375
torch.Size([64, 1419])
-2.198974609375
torch.Size([64, 1419])
-4.45263671875
torch.Size([64, 1419])
-10.339111328125
torch.Size([64, 1419])
-4.425537109375
torch.Size([64, 1419])
-7.180908203125
torch.Size([64, 1419])
1.4951171875
torch.Size([64, 1419])
-2.203857421875
torch.Size([64, 1419])
-1.256591796

-9.61865234375
torch.Size([64, 1419])
-11.5751953125
torch.Size([64, 1419])
-14.579833984375
torch.Size([64, 1419])
-11.3037109375
torch.Size([64, 1419])
-9.56884765625
torch.Size([64, 1419])
-9.003662109375
torch.Size([64, 1419])
-14.8193359375
torch.Size([64, 1419])
-3.076904296875
torch.Size([64, 1419])
-5.0654296875
torch.Size([64, 1419])
-4.175048828125
torch.Size([64, 1419])
-5.078369140625
torch.Size([64, 1419])
-5.256103515625
torch.Size([64, 1419])
-9.3642578125
torch.Size([64, 1419])
-3.50537109375
torch.Size([64, 1419])
-8.75
torch.Size([64, 1419])
-13.175537109375
torch.Size([64, 1419])
-7.43212890625
torch.Size([64, 1419])
-10.42724609375
torch.Size([64, 1419])
-10.028564453125
torch.Size([64, 1419])
-8.668212890625
torch.Size([64, 1419])
-9.188720703125
torch.Size([64, 1419])
-4.603271484375
torch.Size([64, 1419])
-7.333251953125
torch.Size([64, 1419])
-8.166259765625
torch.Size([64, 1419])
-10.08837890625
torch.Size([64, 1419])
-7.94189453125
torch.Size([64, 1419])
-15.4

torch.Size([64, 1419])
-7.731201171875
torch.Size([64, 1419])
-2.03564453125
torch.Size([64, 1419])
-7.09521484375
torch.Size([64, 1419])
-1.1201171875
torch.Size([64, 1419])
-4.076171875
torch.Size([64, 1419])
-7.548095703125
torch.Size([64, 1419])
-2.0322265625
torch.Size([64, 1419])
-0.32568359375
torch.Size([64, 1419])
3.15673828125
torch.Size([64, 1419])
-2.61572265625
torch.Size([64, 1419])
-7.709716796875
torch.Size([64, 1419])
0.845947265625
torch.Size([64, 1419])
-6.25146484375
torch.Size([64, 1419])
-3.975341796875
torch.Size([64, 1419])
-2.820068359375
torch.Size([64, 1419])
-1.001953125
torch.Size([64, 1419])
-6.695556640625
torch.Size([64, 1419])
-4.2890625
torch.Size([64, 1419])
-7.6376953125
torch.Size([64, 1419])
-0.609619140625
torch.Size([64, 1419])
-4.60595703125
torch.Size([64, 1419])
-2.24658203125
torch.Size([64, 1419])
-6.156005859375
torch.Size([64, 1419])
-3.388671875
torch.Size([64, 1419])
-3.37451171875
torch.Size([64, 1419])
-6.671875
torch.Size([64, 1419])


...

torch.Size([64, 1419])
-2.524169921875
torch.Size([64, 1419])
-3.27490234375
torch.Size([64, 1419])
-3.127685546875
torch.Size([64, 1419])
-0.5283203125
torch.Size([64, 1419])
1.4638671875
torch.Size([64, 1419])
0.226806640625
torch.Size([64, 1419])
-1.324462890625
torch.Size([64, 1419])
-1.009765625
torch.Size([64, 1419])
2.24853515625
torch.Size([64, 1419])
0.07177734375
torch.Size([64, 1419])
-1.166259765625
torch.Size([64, 1419])
1.510986328125
torch.Size([64, 1419])
-4.210693359375
torch.Size([64, 1419])
0.97314453125
torch.Size([64, 1419])
-3.9267578125
torch.Size([64, 1419])
0.29541015625
torch.Size([64, 1419])
-0.402587890625
torch.Size([64, 1419])
1.45947265625
torch.Size([64, 1419])
0.3134765625
torch.Size([64, 1419])
4.663330078125
torch.Size([64, 1419])
-1.032470703125
torch.Size([64, 1419])
0.593017578125
torch.Size([64, 1419])
-3.12109375
torch.Size([64, 1419])
3.42041015625
torch.Size([64, 1419])
1.543701171875
torch.Size([64, 1419])
2.238525390625
torch.Size([64, 1419])

torch.Size([64, 1419])
-2.0751953125
torch.Size([64, 1419])
-0.86572265625
torch.Size([64, 1419])
2.876708984375
torch.Size([64, 1419])
-2.820556640625
torch.Size([64, 1419])
-7.352294921875
torch.Size([64, 1419])
3.164306640625
torch.Size([64, 1419])
-3.5380859375
torch.Size([64, 1419])
-2.76513671875
torch.Size([64, 1419])
-3.535888671875
torch.Size([64, 1419])
4.535400390625
torch.Size([64, 1419])
-3.314208984375
torch.Size([64, 1419])
1.31396484375
torch.Size([64, 1419])
0.53955078125
torch.Size([64, 1419])
-4.960693359375
torch.Size([64, 1419])
0.3115234375
torch.Size([64, 1419])
1.628173828125
torch.Size([64, 1419])
-4.8564453125
torch.Size([64, 1419])
0.90380859375
torch.Size([64, 1419])
-3.928955078125
torch.Size([64, 1419])
-3.630859375
torch.Size([64, 1419])
-1.69140625
torch.Size([64, 1419])
1.086181640625
torch.Size([64, 1419])
0.1806640625
torch.Size([64, 1419])
5.221923828125
torch.Size([64, 1419])
-5.6669921875
torch.Size([64, 1419])
2.358642578125
torch.Size([64, 1419])

4.9404296875
torch.Size([64, 1419])
2.281005859375
torch.Size([64, 1419])
-0.22314453125
torch.Size([64, 1419])
-0.748046875
torch.Size([64, 1419])
-1.133056640625
torch.Size([64, 1419])
0.44580078125
torch.Size([64, 1419])
1.42041015625
torch.Size([64, 1419])
3.680419921875
torch.Size([64, 1419])
0.1171875
torch.Size([64, 1419])
0.865234375
torch.Size([64, 1419])
-2.29833984375
torch.Size([64, 1419])
2.917236328125
torch.Size([64, 1419])
-1.7490234375
torch.Size([64, 1419])
2.625
torch.Size([64, 1419])
-2.58056640625
torch.Size([64, 1419])
2.804443359375
torch.Size([64, 1419])
0.845458984375
torch.Size([64, 1419])
-6.029296875
torch.Size([64, 1419])
4.11328125
torch.Size([64, 1419])
0.622802734375
torch.Size([64, 1419])
3.67919921875
torch.Size([64, 1419])
-1.092041015625
torch.Size([64, 1419])
-1.81396484375
torch.Size([64, 1419])
-4.76416015625
torch.Size([64, 1419])
-2.265625
torch.Size([64, 1419])
-0.914306640625
torch.Size([64, 1419])
0.5263671875
torch.Size([64, 1419])
-6.831298

torch.Size([64, 1419])
0.31494140625
torch.Size([64, 1419])
0.41748046875
torch.Size([64, 1419])
-0.15478515625
torch.Size([64, 1419])
-0.3857421875
torch.Size([64, 1419])
4.041259765625
torch.Size([64, 1419])
-0.7783203125
torch.Size([64, 1419])
-4.1201171875
torch.Size([64, 1419])
4.236083984375
torch.Size([64, 1419])
-2.19384765625
torch.Size([64, 1419])
2.15576171875
torch.Size([64, 1419])
-1.66650390625
torch.Size([64, 1419])
0.986572265625
torch.Size([64, 1419])
-2.826171875
torch.Size([64, 1419])
-1.37109375
torch.Size([64, 1419])
2.7666015625
torch.Size([64, 1419])
1.065673828125
torch.Size([64, 1419])
-2.453369140625
torch.Size([64, 1419])
0.828369140625
torch.Size([64, 1419])
0.29248046875
torch.Size([64, 1419])
-3.37646484375
torch.Size([64, 1419])
6.8193359375
torch.Size([64, 1419])
-3.01220703125
torch.Size([64, 1419])
-1.79638671875
torch.Size([64, 1419])
2.813720703125
torch.Size([64, 1419])
6.169677734375
torch.Size([64, 1419])
2.05908203125
torch.Size([64, 1419])
-2.85

KeyboardInterrupt: 